In [5]:
import geopandas as gpd
import json
import ssl 
import geemap
ssl._create_default_https_context = ssl._create_unverified_context
import ee

# Trigger the authentication flow.
# Don't need to run the below as I am alreay authenticated
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [6]:
from typing import Dict, Optional
def ee_geo_compatible(geoJSON_input: Dict):
    geoJSON_output = dict(geoJSON_input)
    try:
        if geoJSON_input["type"]=="Polygon":
            check_coords = geoJSON_input["coordinates"][0][0]
            if len(check_coords)>2:
                print("Forcefully making coordinates compatible ee.Geometry as current len>2:", check_coords)
                new_coords = [[x[0:2] for x in y] for y in geoJSON_input["coordinates"]]
                geoJSON_output["coordinates"] = new_coords
        else:
            print("currently unknown handling for type:", geoJSON_input["type"])
        return geoJSON_output
    except:
        print("Dict passed does not follow geoJSON format")
        return None
            

In [9]:
import geopandas as gpd
from fiona.drvsupport import supported_drivers
# Path to your KML file
kml_file_path = 'townsville.kml'

# Read the KML file using geopandas
supported_drivers['KML'] = 'rw'
my_map = gpd.read_file(kml_file_path, driver='KML')
kml_geo_json=json.loads(my_map.to_json())
new_kml_geo_json = dict(kml_geo_json)
# Initiate with no features
new_kml_geo_json["features"] = []
display("original geojson")
display(kml_geo_json)
for feat in kml_geo_json["features"]:
    new_feat = dict(feat)
    new_feat["geometry"] = ee_geo_compatible(feat["geometry"])
    if new_feat["geometry"]:
        try:
            ee.Geometry(new_feat["geometry"])
            print("succesfully passed to ee.Geometry")
            new_kml_geo_json["features"] += [new_feat]
        except:
            print("Unable to pass into ee.Geometry asses below feature")
            print(new_feat)
            
display("updated geoJSON")
display(new_kml_geo_json)
geojson_fc = ee.FeatureCollection(new_kml_geo_json)
display(geojson_fc.geometry())

'original geojson'

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'Name': 'townsville polygon', 'Description': ''},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[146.6789497574668, -19.23247934166526, 0.0],
      [146.6879911595589, -19.33054650116621, 0.0],
      [146.8302295699071, -19.33087430904637, 0.0],
      [146.8595478481039, -19.26336834562065, 0.0],
      [146.8048945577382, -19.2333878909012, 0.0],
      [146.6789497574668, -19.23247934166526, 0.0]]]}}]}

Forcefully making coordinates compatible ee.Geometry as current len>2: [146.6789497574668, -19.23247934166526, 0.0]
succesfully passed to ee.Geometry


'updated geoJSON'

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'Name': 'townsville polygon', 'Description': ''},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[146.6789497574668, -19.23247934166526],
      [146.6879911595589, -19.33054650116621],
      [146.8302295699071, -19.33087430904637],
      [146.8595478481039, -19.26336834562065],
      [146.8048945577382, -19.2333878909012],
      [146.6789497574668, -19.23247934166526]]]}}]}

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Collection.geometry",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection",
          "arguments": {
            "features": {
              "arrayValue": {
                "values": [
                  {
                    "functionInvocationValue": {
                      "functionName": "Feature",
                      "arguments": {
                        "geometry": {
                          "functionInvocationValue": {
                            "functionName": "GeometryConstructors.Polygon",
                            "arguments": {
                              "coordinates": {
                                "constantValue": [
                                  [
                                    [
                                      146.6789497574668,
                                      -19.23247934166526
                                    ],
                                    [
                                      146.6879911595589,
                                      -19.33054650116621
                                    ],
                                    [
                                      146.8302295699071,
                                      -19.33087430904637
                                    ],
                                    [
                                      146.8595478481039,
                                      -19.26336834562065
                                    ],
                                    [
                                      146.8048945577382,
                                      -19.2333878909012
                                    ],
                                    [
                                      146.6789497574668,
                                      -19.23247934166526
                                    ]
                                  ]
                                ]
                              }
                            }
                          }
                        },
                        "metadata": {
                          "constantValue": {
                            "Description": "",
                            "Name": "townsville polygon",
                            "system:index": "0"
                          }
                        }
                      }
                    }
                  }
                ]
              }
            }
          }
        }
      }
    }
  }
})

In [31]:
# Initialise the Geemap
m = geemap.Map()
poly = geojson_fc.geometry()

m.centerObject(poly, 12)
# import land cover maps 2009, 2015, 2019, 2021

# 2009 GlobCover
dataset = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(
    'landcover'
)
visualization = {
}
m.add_ee_layer(dataset.clip(poly), visualization, '2009_Landcover', shown=False)
m.add_legend(title='2009_Landcover', builtin_legend='GLOBCOVER', layer_name='2009_Landcover')

# 2015 Copernicus
visualization = {
    'bands': ['discrete_classification'],
}
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2015').select(
    'discrete_classification'
)
m.add_ee_layer(dataset.clip(poly), visualization, '2015_Landcover', shown=False)
m.add_legend(title='2015_Landcover', builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global', layer_name='2015_Landcover')

# 2019 Copernicus
dataset = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select(
    'discrete_classification'
)
visualization = {
    'bands': ['discrete_classification'],
}
m.add_ee_layer(dataset.clip(poly), visualization, '2019_Landcover', shown=False)
m.add_legend(title='2019_Landcover', builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global', layer_name='2019_Landcover')

# 2021 ESA World Cover
dataset = ee.ImageCollection('ESA/WorldCover/v200').first()
visualization = {
    'bands': ['Map'],
}
m.add_ee_layer(dataset.clip(poly), visualization, '2021_Landcover', shown=False)
m.add_legend(title='2021_Landcover', builtin_legend='ESA_WorldCover', layer_name='2021_Landcover')

# Add historical satalite views 
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

for yr in ['2017','2019', '2021']:
        
    dataset = (
        ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterDate(f'{yr}-01-01', f'{yr}-12-31')
        # Pre-filter to get less cloudy granules.
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(mask_s2_clouds)
    )

    m.add_layer(dataset.median().clip(poly), rgb_vis, f'{yr}_Satalite', shown=False)

display(m)

Map(center=[-19.281796805191107, 146.76263396612705], controls=(WidgetControl(options=['position', 'transparen…

In [20]:
# Calculate the forest cover % for the polygon provided

In [32]:
if m.draw_features:
    new_drawn_fc = ee.FeatureCollection(m.draw_features)
    new_poly = new_drawn_fc.geometry()
    # 2021 ESA World Cover
    dataset = ee.ImageCollection('ESA/WorldCover/v200').first()
    visualization = {
        'bands': ['Map'],
    }
    m.add_ee_layer(dataset.clip(new_poly), visualization, 'drawn', shown=False)
    # display(m)
else:
    m.remove_layer('drawn')

In [34]:
land_cover_dict = {
    "Tree cover":{
        "color": "#006400",
        "value": 10, 
    },
    "Shrubland":{
        "color": "#ffbb22",
        "value": 20, 
    },
    "Grassland":{
        "color": "#ffff4c",
        "value": 30, 
    },
    "Cropland":{
        "color": "#f096ff",
        "value": 40, 
    },
    "Built-up":{
        "color": "#fa0000",
        "value": 50, 
    },
    "Bare / sparse vegetation":{
        "color": "#b4b4b4",
        "value": 60, 
    },
    "Snow and ice":{
        "color": "#f0f0f0",
        "value": 70, 
    },
    "Permanent water bodies":{
        "color": "#0064c8",
        "value": 80, 
    },
    "Herbaceous wetland":{
        "color": "#0096a0",
        "value": 90, 
    },
    "Mangroves":{
        "color": "#00cf75",
        "value": 95, 
    },
    "Moss and lichen":{ 
        "color": "#fae6a0",
        "value": 100, 
    },
}


In [56]:
# for key, vals in land_cover_dict.items():
# Create a pixel area image. Pixel values are square meters based on
# a given CRS and scale (or CRS transform).
pixel_area = ee.Image.pixelArea()

# The default projection is WGS84 with 1-degree scale.
display('Pixel area default projection', pixel_area.projection())

# The "area" band produced by the `pixel_area` function can be useful for
# calculating the area of a certain condition of another image. For example,
# here we use the sum reducer to determine the area above 2250m in the North
# Cascades poly, according to a 30m digital elevation model.

# Import a DEM and subset the "elevation" band.
elev =  ee.ImageCollection('ESA/WorldCover/v100').first().select('Map')

# Define a high elevation mask where pixels with elevation greater than 2250m
# are set to 1, otherwise 0.
high_elev_mask = elev.clip(poly).eq(10)

# Apply the high elevation mask to the pixel area image.
high_elev_area = pixel_area.updateMask(high_elev_mask)

# Sum the area of high elevation pixels in the North Cascades poly.
area = high_elev_area.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=poly,
    crs=elev.projection(),  # DEM coordinate reference system
    crsTransform=elev.projection().getInfo()['transform'],  # DEM grid alignment
    maxPixels=1e10,
)

# Fetch the summed area property from the resulting dictionary and convert
# square meters to square kilometers.
square_meters = area.getNumber('area')
square_kilometers = square_meters.divide(1e6)

display('Square meters above 2250m elevation', square_meters.getInfo())
display('Square kilometers above 2250m elevation', square_kilometers.getInfo())

total_area = poly.area()
display('total area', total_area)
display(square_meters.getInfo()/total_area.getInfo()*100)

'Pixel area default projection'

'Square meters above 2250m elevation'

33387596.59492597

'Square kilometers above 2250m elevation'

33.38759659492597

'total area'

18.59448783882218

In [72]:

# elev =  ee.ImageCollection('ESA/WorldCover/v100').first().select('Map')
elev = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2015').select(
    'discrete_classification'
)

areaImage = ee.Image.pixelArea().addBands(elev.clip(poly))

areas = areaImage.reduceRegion(
    reducer = ee.Reducer.sum().group(
        groupField= 1,
        groupName= 'class',
    ),
    geometry= poly,
    scale= 1,
    maxPixels= 1e10
    )
 
# print(areas)

classAreas = ee.List(areas.get('groups'))

def area_sum_by_class_func(item):
    areaDict = ee.Dictionary(item)
    classNumber = ee.Number(areaDict.get('class')).format()
    area = ee.Number(areaDict.get('sum'))
    return ee.List([classNumber, area])
    
 
classAreaLists = classAreas.map(area_sum_by_class_func)
 
result = ee.Dictionary(classAreaLists.flatten())
print(result.getInfo())

{'112': 3389396.406239998, '114': 7204428.669131251, '116': 310196.01696294546, '122': 133244.30223357677, '124': 25516154.070521645, '126': 8592277.856392391, '20': 761373.0806720479, '200': 9001257.620573804, '30': 35213468.519375294, '40': 450415.0890374962, '50': 86544067.0144705, '60': 895136.2773757577, '80': 796059.5632449699, '90': 206518.7984795783}


In [73]:
display(result.getInfo())
tot_perc = 0
poly_area = poly.area().getInfo()
for val, area in result.getInfo().items():
    perc =  area/poly_area*100
    tot_perc += perc
    print('class:', val, 'Area:', area, 'Perc:', perc, 'Cum_Perc:', tot_perc)

{'112': 3389396.406239998,
 '114': 7204428.669131251,
 '116': 310196.01696294546,
 '122': 133244.30223357677,
 '124': 25516154.070521645,
 '126': 8592277.856392391,
 '20': 761373.0806720479,
 '200': 9001257.620573804,
 '30': 35213468.519375294,
 '40': 450415.0890374962,
 '50': 86544067.0144705,
 '60': 895136.2773757577,
 '80': 796059.5632449699,
 '90': 206518.7984795783}

class: 112 Area: 3389396.406239998 Perc: 1.8876498066457181 Cum_Perc: 1.8876498066457181
class: 114 Area: 7204428.669131251 Perc: 4.012348145304406 Cum_Perc: 5.899997951950124
class: 116 Area: 310196.01696294546 Perc: 0.17275685144539998 Cum_Perc: 6.072754803395524
class: 122 Area: 133244.30223357677 Perc: 0.07420748451989867 Cum_Perc: 6.146962287915422
class: 124 Area: 25516154.070521645 Perc: 14.210660992289966 Cum_Perc: 20.357623280205388
class: 126 Area: 8592277.856392391 Perc: 4.785280235857111 Cum_Perc: 25.1429035160625
class: 20 Area: 761373.0806720479 Perc: 0.42402999716111683 Cum_Perc: 25.566933513223617
class: 200 Area: 9001257.620573804 Perc: 5.013052523382335 Cum_Perc: 30.579986036605952
class: 30 Area: 35213468.519375294 Perc: 19.61136706215566 Cum_Perc: 50.19135309876161
class: 40 Area: 450415.0890374962 Perc: 0.250848780675712 Cum_Perc: 50.44220187943732
class: 50 Area: 86544067.0144705 Perc: 48.19881530098953 Cum_Perc: 98.64101718042684
class: 60 Area: 895136.2773757577

In [64]:

def class_val_dict(val_list, class_name_list):
    output_dict = {}
    if len(val_list)==len(class_name_list):
        for i in range(len(val_list)):
            output_dict[f"{val_list[i]}"]=class_name_list[i]
    return output_dict

new_dict = class_val_dict(elev.get("Map_class_values").getInfo(), elev.get("Map_class_names").getInfo())
new_dict

{'10': 'Tree cover',
 '20': 'Shrubland',
 '30': 'Grassland',
 '40': 'Cropland',
 '50': 'Built-up',
 '60': 'Bare / sparse vegetation',
 '70': 'Snow and ice',
 '80': 'Permanent water bodies',
 '90': 'Herbaceous wetland',
 '95': 'Mangroves',
 '100': 'Moss and lichen'}